# Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones
---

<p style="text-align: center;">
<img src=http://www2.famaf.unc.edu.ar/~efernandez/egeo/img/logos/famaf.jpg width=40%>
</p>

 Universidad Nacional de Córdoba

---

## Practico: Aprendizaje No Supervisado
## 1. Preprocesamiento

En la siguiente notebook se presentará las operaciones de preprocesamiento que deben ser aplicadas previo a entrenar los modelos de aprendizaje no supervisado

### Importación de librerías

In [1]:
import pandas as pd 
import numpy as np
import warnings
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

warnings.filterwarnings('ignore')

### Lectura del dataset 

In [2]:
df = pd.read_parquet('cupones_no_supervisado.parquet')
df

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,EMPDE COMERCIO,SECUNDARIOS,Propia,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,61,92,0
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,SIN DATOS,None,None,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,29,2,0
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,SIN DATOS,PRIMARIOS,Otros,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,28,95,0
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,EMPDE COMERCIO,SECUNDARIOS,Propia,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,40,151,0
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,EMPDE COMERCIO,UNIVERSITARIOS,Alquilada,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,36,87,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124311,0006c15ca823454b68c189da1344d9d7317,0,064194852,NARANJA PLUS RESISTENCIA II,821734,0,PC,1,3,9,...,JUBILADO,SECUNDARIOS,Propia,202105,13210.67,NARANJA PLUS RESISTENCIA II,Sector_No_Operativo,66,105,0
124312,0000ab27a0ed815f947df8bcb834ff97975,0,063299398,PLAN DE CUOTAS SGO DEL ESTERO,152686,0,PC,1,3,3,...,ADMINISTRATIVO,SECUNDARIOS,Propia,202105,121.73,PLAN DE CUOTAS SGO DEL ESTERO,Sector_Empleado_Comercio,74,140,0
124313,0000ab27a0ed815f947df8bcb834ff97975,0,063299398,PLAN DE CUOTAS SGO DEL ESTERO,152687,0,PC,1,3,3,...,ADMINISTRATIVO,SECUNDARIOS,Propia,202105,12284.16,PLAN DE CUOTAS SGO DEL ESTERO,Sector_Empleado_Comercio,74,140,0
124314,000986998f686d538f893956e736fedf611,3,350071053,20 DE MAYO SRL,70,0,PL,1,0,1,...,OPERARIO,SECUNDARIOS,Propia,202002,1040.00,20 DE MAYO SRL,Sector_Operativo,45,261,3


### 1.1 Imputación de datos nulos

In [3]:
df.isna().sum()

dni_titular_movimiento                      0
moneda_movimiento                           0
id_comercio_movimiento                      0
nombre_comercio_histo                       0
numero_cupon_movimiento                     0
debito_credito_movimiento                   0
producto_naranja_movimiento                 0
codigo_empresa_movimiento                   0
tipo_producto_tarjeta_movimiento            0
plan_movimiento                             0
fecha_vto_cupon_movimiento                  0
fecha_presentacion_movimiento               0
fecha_cupon_movimiento                      0
fecha_carga_sistema_movimiento          40715
monto_compra_movimiento                     0
importe_cuota_movimiento                    0
interes_movimiento                          0
cargo_adm_seguro_movimiento                 0
cargo_otorgamiento_movimiento               0
cargo_seguro_vida_movimiento                0
cargo_administrativo_movimiento             0
seleccionado_ng                   

#### Fecha carga sistema movimiento

In [4]:
print('fecha_cupon vs fecha_carga:')
df["fecha_cupon_movimiento"].dt.date.corr(
    df["fecha_carga_sistema_movimiento"].dt.date, method = 'spearman', min_periods = 1)

fecha_cupon vs fecha_carga:


0.9984405676446135

Al calcular su correlación con la columna **fecha_cupon_movimiento**, podemos observar que tienen una relación lineal fuerte, por lo cual decidimos no tener en cuenta esta columna, y si utilizar fecha_cupon_movimiento en su lugar.

In [5]:
df.drop('fecha_carga_sistema_movimiento', axis=1, inplace=True)

#### Tipo_prestamo_movimiento, Nombre_local_histo y Fecha_extraccion_movimiento 

Las variables tipo_prestamo_movimiento, nombre_local_histo y fecha_extraccion_movimiento poseen una gran cantidad de valores nulos (fecha extracción tiene todos sus valores nulos), por lo cuál la mejor decisión es no tenerlas en cuenta.

In [6]:
df.drop(columns = ['tipo_prestamo_movimiento','nombre_local_histo','fecha_extraccion_movimiento'], inplace=True)

#### Sexo

In [7]:
(df[df['sexo_descripcion'].isna()])['sexo_descripcion']

220       None
378       None
474       None
485       None
634       None
          ... 
124145    None
124213    None
124214    None
124225    None
124226    None
Name: sexo_descripcion, Length: 647, dtype: object

Procedemos a imputar con 'Sin Datos' a los valores faltantes en la columna sexo_descripcion.

In [8]:
columna = ['sexo_descripcion']
df['sexo_descripcion']= df.sexo_descripcion.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [9]:
#df['sexo_descripcion'] = df['sexo_descripcion'].astype(str)
df['sexo_descripcion'].value_counts()

Mujer        65541
Hombre       58128
Sin Datos      647
Name: sexo_descripcion, dtype: int64

#### Datos faltantes Nivel de Estudio

In [10]:
(df[df['nivel_estudio_descripcion_histo'].isna()])['nivel_estudio_descripcion_histo']

1         None
9         None
27        None
30        None
37        None
          ... 
124171    None
124179    None
124181    None
124217    None
124218    None
Name: nivel_estudio_descripcion_histo, Length: 4393, dtype: object

Habiendo analizado en los trabajos anteriores que la pérdida de datos de nivel de estudio también es sistemática, es decir no es posible recuperar este dato, procedemos a imputar dichas filas con Sin datos. 

In [11]:
columna = ['nivel_estudio_descripcion_histo']
df['nivel_estudio_descripcion_histo']= df.nivel_estudio_descripcion_histo.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [12]:
df.nivel_estudio_descripcion_histo.value_counts()

SECUNDARIOS       66356
PRIMARIOS         30472
TERCIARIOS        14332
UNIVERSITARIOS     8763
Sin Datos          4393
Name: nivel_estudio_descripcion_histo, dtype: int64

#### Datos faltantes Vivienda

In [13]:
(df[df['rel_vivienda_descripcion_histo'].isna()])['rel_vivienda_descripcion_histo']

1         None
7         None
9         None
15        None
27        None
          ... 
124276    None
124279    None
124280    None
124281    None
124282    None
Name: rel_vivienda_descripcion_histo, Length: 13740, dtype: object

Al igual que el caso anterior, la pérdida de datos de vivienda también es sistemática, es decir no es posible recuperar este dato, procedemos a imputar dichas filas con Sin datos. 

In [14]:
columna = ['rel_vivienda_descripcion_histo']
df["rel_vivienda_descripcion_histo"]= df.rel_vivienda_descripcion_histo.fillna(value=np.nan)
const_imputer = SimpleImputer(missing_values= np.nan, strategy='constant',fill_value="Sin Datos") 
df.loc[ : , columna] = pd.DataFrame(const_imputer.fit_transform(df.loc[:][columna]) , columns = columna)

In [15]:
df.rel_vivienda_descripcion_histo.value_counts()

Propia         85313
Sin Datos      13740
Otros          10653
De familiar    10473
Alquilada       4137
Name: rel_vivienda_descripcion_histo, dtype: int64

#### Fecha de nacimiento y edad 

In [16]:
(df[df['fecha_nacimiento'].isna()])

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat
220,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202008,0.98,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,237,0
378,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202008,0.98,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,237,0
474,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202009,66.48,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,238,0
634,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202009,66.48,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,238,0
768,000650e8144e3a42d47a21aee9e48f34804,0,020100936,CARGO POR GESTION DE COBRANZA,999990,0,PL,1,0,1,...,JUBILADO,PRIMARIOS,Propia,202010,2.32,CARGO POR GESTION DE COBRANZA,Sector_No_Operativo,<NA>,239,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122068,000cde1ad5d8114e7a5dd977930b8835659,3,112000973,LADY STORK STORK MAN,6292,0,PL,1,3,1,...,SIN DATOS,SECUNDARIOS,Propia,202007,5244.57,LADY STORK STORK MAN,Sector_Sin_Datos,<NA>,17,1
124213,000cde1ad5d8114e7a5dd977930b8835659,0,112101030,EL EMPORIO DE LAS GOLOSINAS,1665,0,PL,1,3,1,...,SIN DATOS,SECUNDARIOS,Propia,202007,515.40,EL EMPORIO DE LAS GOLOSINAS,Sector_Sin_Datos,<NA>,17,1
124214,000cde1ad5d8114e7a5dd977930b8835659,0,112101030,EL EMPORIO DE LAS GOLOSINAS,1665,0,PL,1,3,1,...,SIN DATOS,SECUNDARIOS,Propia,202007,515.40,EL EMPORIO DE LAS GOLOSINAS,Sector_Sin_Datos,<NA>,17,1
124225,000650e8144e3a42d47a21aee9e48f34804,3,112142903,ROSARIO VISION OPTICAS,84,0,PL,1,3,1,...,JUBILADO,PRIMARIOS,Propia,202007,2536.23,ROSARIO VISION OPTICAS,Sector_No_Operativo,<NA>,236,1


Habiendo analizado en trabajos anteriores, que la falta de la fecha de nacimiento es una pérdida sistemática, no podemos recuperar dicho dato de una forma confiable. Debido a que en los modelos de aprendizaje no supervisado cobra mucha importancia la descripción de los individuos a los efectos de poder agrupar los similares, creemos conveniente no considerar aquellas filas en las cuales no tenemos el dato de la fecha de nacimiento, y por lo tanto tampoco su edad. 

In [17]:
df = df[(~df['fecha_nacimiento'].isnull())]
df.shape

(124010, 44)

#### Datos Faltantes Geograficos de los clientes

In [18]:
col_geo = ['domicilio_codigo_postal', 'pais', 'provincia', 'ciudad', 'domicilio_barrio']
df[col_geo].isna().sum()

domicilio_codigo_postal     651
pais                       3092
provincia                  3118
ciudad                     3579
domicilio_barrio            839
dtype: int64

##### Reimputación con datos geográficos presentes en el dataset

En primer lugar, recuperaremos las filas donde haya algún dato geográfico presente que permita imputar los faltantes. 

In [19]:
pd.DataFrame({'Provincias': df['provincia'].unique()})\
    .sort_values(by='Provincias',ascending=True)\
    .reset_index().drop(columns=['index'])

,Provincias
0,BUENOS AIRES
1,CAPITAL FEDERAL
2,CATAMARCA
3,CHACO
4,CHUBUT
5,CORDOBA
6,CORRIENTES
7,ENTRE RIOS
8,FORMOSA
9,JUJUY


Tenemos un total de 24 provincias, el resto son registros NaN.

In [20]:
df[df['domicilio_codigo_postal'].str.len()<4][['pais','provincia','ciudad','domicilio_codigo_postal']].drop_duplicates()

,pais,provincia,ciudad,domicilio_codigo_postal
24863,Argentina,None,None,0
58603,Argentina,TUCUMAN,CAMPO HERRERA,400


Observamos que tenemos datos erróneos de código postal con valores de 0 y 400. 

Analizaremos si podemos recuperar el código postal de la localidad de **Campo Herrera**. El Código Postal en 0 lo imputaremos por NaN.

In [21]:
print('Cantidad de filas con Cero:', len(df[df['domicilio_codigo_postal']=='0']))

Cantidad de filas con Cero: 26


In [22]:
df['domicilio_codigo_postal'] = df['domicilio_codigo_postal'].replace('0',np.nan)
df[df['domicilio_codigo_postal']=='0']

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,cargo_descripcion_histo,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat


In [23]:
df['ciudad'] = df['ciudad'].str.strip()
print('Cantidad de filas con 400 en el CP:',len(df[df['ciudad'].isin(['CAMPO HERRERA'])]))
df[df['ciudad'].isin(['CAMPO HERRERA'])][['pais','provincia','ciudad','domicilio_codigo_postal']].drop_duplicates()

Cantidad de filas con 400 en el CP: 20


,pais,provincia,ciudad,domicilio_codigo_postal
58603,Argentina,TUCUMAN,CAMPO HERRERA,400


Obteniendo el dato de una fuente externa, vamos a reemplazar el código postal de la ciudad Campo Herrera en la provincia Tucumán por el valor correcto.

Fuente: https://codigo-postal.co/argentina/tucuman/campo-herrera/

In [24]:
df.loc[df['ciudad'].isin(['CAMPO HERRERA']),'domicilio_codigo_postal'] = '4105'
print('Cantidad de filas con 400 en el CP:',len(df[df['ciudad'].isin(['CAMPO HERRERA'])]))
df[df['ciudad'].isin(['CAMPO HERRERA'])][['pais','provincia','ciudad','domicilio_codigo_postal']].drop_duplicates()

Cantidad de filas con 400 en el CP: 20


,pais,provincia,ciudad,domicilio_codigo_postal
58603,Argentina,TUCUMAN,CAMPO HERRERA,4105


In [25]:
df[df['pais'].isnull()][['pais','provincia','ciudad','domicilio_codigo_postal','domicilio_barrio']].isna().sum()

pais                       3092
provincia                  3092
ciudad                     3092
domicilio_codigo_postal     651
domicilio_barrio            651
dtype: int64

Observamos que cuando **pais** es nulo, tambien lo son las variables **provincia** y **ciudad**. Primero vemos si existen datos dentro del dataset con los cuales imputar. 

Procedemos a recuperar los datos de **provincia** y **ciudad** a través de **domicilio_codigo_postal** dentro de nuestro dataset.

In [26]:
cp_ciu_nan = df[(df['domicilio_codigo_postal'].isna()!=True) & (df['ciudad'].isna()==True)]['domicilio_codigo_postal'].unique()
df[df['domicilio_codigo_postal'].isin(cp_ciu_nan)][['domicilio_codigo_postal','ciudad']]\
.drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,ciudad
5899,1130,None
3027,1663,SAN MIGUEL
2687,1663,None
2271,2144,None
58610,2506,None
58684,2705,None
17839,2812,None
23486,3230,None
58620,3240,None
116,3300,POSADAS


In [27]:
df.loc[:,'ciudad'] = df['ciudad'].str.upper()
df_dp_cd = df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['ciudad'].isna()==False)]\
            [['domicilio_codigo_postal','ciudad']]\
            .drop_duplicates()\
            .sort_values(by=['domicilio_codigo_postal'])
df_dp_cd = df_dp_cd.rename(columns={'ciudad': 'ciudad_0', 'domicilio_codigo_postal': 'cp'})
df_dp_cd

,cp,ciudad_0
3027,1663,SAN MIGUEL
116,3300,POSADAS
26,3500,RESISTENCIA
870,3503,LA LEONESA
105,4000,SAN MIGUEL DE TUCUMAN
2024,4000,TUCUMAN
174,4101,SAN MIGUEL DE TUCUMAN
795,4101,LAS TALITASTUCUMAN
89,4400,SALTA
100,5000,CORDOBA


Realizamos merge sobre el dataframe creado, e imputamos los datos de ciudad.

In [28]:
df = df.merge(df_dp_cd.drop_duplicates(subset=['cp'], keep='first'), 
              how='left',
              left_on = 'domicilio_codigo_postal', 
              right_on = 'cp')\
            .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,ciudad_0
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,SECUNDARIOS,Propia,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,61,92,0,NaN
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Sin Datos,Sin Datos,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,29,2,0,NaN
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,PRIMARIOS,Otros,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,28,95,0,NaN
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,SECUNDARIOS,Propia,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,40,151,0,NaN
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,UNIVERSITARIOS,Alquilada,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,36,87,0,NaN


In [29]:
# Sustituimos los valores nulos de la columna provincia (provincia_x) por las provincias (en mayus) de la columna
# provincia creada en la anterior unión.
df.loc[df['ciudad'].isnull(), 'ciudad'] = df['ciudad_0'].str.upper()

df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['ciudad'].isna()==True)]\
    [['domicilio_codigo_postal','ciudad','ciudad_0']].drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,ciudad,ciudad_0
5879,1130,NaN,NaN
2253,2144,NaN,NaN
58498,2506,NaN,NaN
58571,2705,NaN,NaN
17785,2812,NaN,NaN
23426,3230,NaN,NaN
58508,3240,NaN,NaN
2094,4146,NaN,NaN
58516,5019,NaN,NaN
491,5101,NaN,NaN


In [30]:
df = df.drop(columns=['ciudad_0'])

Procedemos a imputar **provincia** de la misma manera que **ciudad**.

In [31]:
cp_prov_nan = df[(df['domicilio_codigo_postal'].isna()!=True) & (df['provincia'].isna()==True)]['domicilio_codigo_postal'].unique()
df[df['domicilio_codigo_postal'].isin(cp_prov_nan)][['domicilio_codigo_postal','provincia']]\
.drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,provincia
5879,1130,None
3007,1663,BUENOS AIRES
2667,1663,None
2253,2144,None
58571,2705,None
17785,2812,None
23426,3230,None
116,3300,MISIONES
24667,3300,None
26,3500,CHACO


In [32]:
df.loc[:,'provincia'] = df['provincia'].str.upper()
df_dp_pv = df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['provincia'].isna()==False)]\
            [['domicilio_codigo_postal','provincia']]\
            .drop_duplicates()\
            .sort_values(by=['domicilio_codigo_postal'])
df_dp_pv = df_dp_pv.rename(columns={'provincia': 'provincia_0', 'domicilio_codigo_postal': 'cp'})
df_dp_pv

,cp,provincia_0
3007,1663,BUENOS AIRES
58498,2506,SANTA FE
58508,3240,ENTRE RIOS
116,3300,MISIONES
26,3500,CHACO
865,3503,CHACO
105,4000,TUCUMAN
174,4101,TUCUMAN
77,4400,SALTA
100,5000,CORDOBA


In [33]:
df = df.merge(df_dp_pv.drop_duplicates(subset=['cp'], keep='first'), 
              how='left',
              left_on = 'domicilio_codigo_postal', 
              right_on = 'cp')\
            .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,provincia_0
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,SECUNDARIOS,Propia,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,61,92,0,NaN
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Sin Datos,Sin Datos,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,29,2,0,NaN
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,PRIMARIOS,Otros,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,28,95,0,NaN
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,SECUNDARIOS,Propia,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,40,151,0,NaN
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,UNIVERSITARIOS,Alquilada,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,36,87,0,NaN


In [34]:
# Sustituimos los valores nulos de la columna provincia (provincia_0) por las provincias (en mayus) de la columna
# provincia creada en la anterior unión.

df.loc[df['provincia'].isnull(), 'provincia'] = df['provincia_0'].str.upper()

df[(df['domicilio_codigo_postal'].isin(cp_ciu_nan)) & (df['provincia'].isna()==True)]\
    [['domicilio_codigo_postal','provincia','provincia_0']].drop_duplicates().sort_values(by=['domicilio_codigo_postal'])

,domicilio_codigo_postal,provincia,provincia_0
5879,1130,NaN,NaN
2253,2144,NaN,NaN
58571,2705,NaN,NaN
17785,2812,NaN,NaN
23426,3230,NaN,NaN
2094,4146,NaN,NaN
1472,5613,NaN,NaN
2704,8322,NaN,NaN
5446,9015,NaN,NaN


In [35]:
df = df.drop(columns=['provincia_0'])

In [36]:
df[col_geo].isna().sum()

domicilio_codigo_postal     677
pais                       3092
provincia                  1535
ciudad                     1818
domicilio_barrio            839
dtype: int64

Como podemos observar, existen datos nulos en **provincia** y **ciudad** que persistieron a la imputación anterior, por lo que decidimos buscar en un CSV externo los datos de Códigos Postales, Localidades y Provincias de Argentina.

##### Imputación con dataset externo

In [37]:
df_cp = pd.read_csv('https://raw.githubusercontent.com/JIBarrionuevoGaltier/localidades_AR/master/localidades_cp_maestro.csv')
df_cp.head()

,provincia,id,localidad,cp,id_prov_mstr
0,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,NaN,2
1,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1144.0,2
2,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1145.0,2
3,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1146.0,2
4,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1147.0,2


In [38]:
# Realizamos una curación de datos sobre este dataset.

df_cp.cp = df_cp[df_cp['cp'].isna()!=True]['cp'].apply(int).apply(str)
df_cp.head()

,provincia,id,localidad,cp,id_prov_mstr
0,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,NaN,2
1,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1144,2
2,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1145,2
3,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1146,2
4,Ciudad Autonoma de Buenos Aires,5001,Ciudad Autonoma de Buenos Aires,1147,2


Observamos si todos los Códigos Postales de nuestro DataFrame original se encuentran en los objetos del merge.

In [39]:
df_cp = df_cp[~df_cp['cp'].isnull()]
df_cp.isna().sum()

provincia       0
id              0
localidad       0
cp              0
id_prov_mstr    0
dtype: int64

In [40]:
pd.DataFrame({'Provincias': df_cp['provincia'].unique()})\
    .sort_values(by='Provincias',ascending=True)\
    .reset_index().drop(columns=['index'])

,Provincias
0,Buenos Aires
1,Catamarca
2,Chaco
3,Chubut
4,Ciudad Autonoma de Buenos Aires
5,Cordoba
6,Corrientes
7,Entre Rios
8,Formosa
9,Jujuy


In [41]:
df_cp.loc[df_cp['provincia'] == 'Ciudad Autonoma de Buenos Aires', 'provincia'] = 'CAPITAL FEDERAL'
df_cp.loc[df_cp['provincia'] == 'Santiago del Estero', 'provincia'] = 'SGO DEL ESTERO'
df_cp['provincia'].unique()

array(['CAPITAL FEDERAL', 'Buenos Aires', 'Catamarca', 'Cordoba',
       'Corrientes', 'Chaco', 'Chubut', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen',
       'Rio Negro', 'Salta', 'San Juan', 'San Luis', 'Santa Cruz',
       'Santa Fe', 'SGO DEL ESTERO', 'Tucuman', 'Tierra del Fuego'],
      dtype=object)

In [42]:
df_cp.loc[:,'provincia'] = df_cp['provincia'].str.upper()
df_cp.loc[:,'localidad'] = df_cp['localidad'].str.upper()
df_cp[['provincia','localidad']].drop_duplicates()

,provincia,localidad
1,CAPITAL FEDERAL,CIUDAD AUTONOMA DE BUENOS AIRES
456,BUENOS AIRES,COLONIA VELEZ
457,BUENOS AIRES,SPURR
458,BUENOS AIRES,SPERONI
459,BUENOS AIRES,SPERATTI
...,...,...
23234,TIERRA DEL FUEGO,ESTANCIA SAN JULIO
23235,TIERRA DEL FUEGO,ESTANCIA SAN JUSTO
23236,TIERRA DEL FUEGO,ESTANCIA SAN MARTIN
23237,TIERRA DEL FUEGO,ESTANCIA RIO EWAN


In [43]:
df_cp = df_cp.rename(columns={'provincia': 'provincia_0'})
df_cp.head()

,provincia_0,id,localidad,cp,id_prov_mstr
1,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1144,2
2,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1145,2
3,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1146,2
4,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1147,2
5,CAPITAL FEDERAL,5001,CIUDAD AUTONOMA DE BUENOS AIRES,1148,2


In [44]:
df.shape

(124010, 44)

Procedemos a hacer el merge de los datasets, a través de la columna cp (Codigo Postal)

In [45]:
# Unimos por cp, eliminamos duplicados, conservamos solo cps del dataframe original
df = df.merge(df_cp[['provincia_0','localidad','cp']].drop_duplicates(subset=['cp'], keep='first'), 
                           how='left',
                           left_on = 'domicilio_codigo_postal', 
                           right_on = 'cp')\
                    .drop(columns= ['cp'])
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,provincia_0,localidad
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Propia,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,61,92,0,BUENOS AIRES,LA TABLADA
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Sin Datos,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,29,2,0,MENDOZA,VILLA NUEVA
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Otros,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,28,95,0,BUENOS AIRES,SANTA TERESITA
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Propia,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,40,151,0,SANTA FE,SAN LORENZO
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Alquilada,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,36,87,0,CAPITAL FEDERAL,CIUDAD AUTONOMA DE BUENOS AIRES


In [46]:
df.loc[df['provincia'].isnull(), 'provincia'] = df['provincia_0'].str.upper()
df.loc[df['ciudad'].isnull(), 'ciudad'] = df['localidad'].str.upper()

In [47]:
df = df.drop(columns=['provincia_0','localidad'])

In [48]:
df[col_geo].isna().sum()

domicilio_codigo_postal     677
pais                       3092
provincia                   677
ciudad                      697
domicilio_barrio            839
dtype: int64

##### Conclusiones y decisiones

* La columna **pais** tiene una sola categoria: 'Argentina'. Sabiendo que todas los registros de la columna **provincia** que tienen datos pertenecen al territorio argentino, se puede descartar ya que no aporta información.
* La columna **domicilio_barrio** no aporta mayor información a la que ya se obtiene con el resto de las columnas geograficas, por lo que tambien se descarta. 
* Los filas con valores faltantes en las columnas **domicilio_codigo_postal**, **provincia** y **ciudad** se eliminan ya que no se pueden recuperar. 

In [49]:
df.drop(columns=['pais','domicilio_barrio'], inplace=True)

In [50]:
df = df[(~df['domicilio_codigo_postal'].isnull())]
df.shape

(123333, 42)

In [51]:
df.isna().sum()

dni_titular_movimiento                  0
moneda_movimiento                       0
id_comercio_movimiento                  0
nombre_comercio_histo                   0
numero_cupon_movimiento                 0
debito_credito_movimiento               0
producto_naranja_movimiento             0
codigo_empresa_movimiento               0
tipo_producto_tarjeta_movimiento        0
plan_movimiento                         0
fecha_vto_cupon_movimiento              0
fecha_presentacion_movimiento           0
fecha_cupon_movimiento                  0
monto_compra_movimiento                 0
importe_cuota_movimiento                0
interes_movimiento                      0
cargo_adm_seguro_movimiento             0
cargo_otorgamiento_movimiento           0
cargo_seguro_vida_movimiento            0
cargo_administrativo_movimiento         0
seleccionado_ng                         0
codigo_contable_movimiento              0
local_venta_producto                    0
marca_debito_automatico           

#### 1.2 Eliminación de filas duplicadas 

Habiendo imputados todos los valores faltantes, eliminamos aquellas filas que se encuentran duplicadas.

In [52]:
df = df[~df.duplicated()]

In [53]:
df.shape

(64897, 42)

#### 1.3 Valores Atípicos

En este punto, se remite al análisis efectuado en el Trabajo Práctico N° 2 punto 3.2. A diferencia de lo trabajado con los modelos de Aprendizaje Supervisado, en este caso consideramos conveniente quedarnos con aquellas filas con consumos pocos frecuentes, al igual que edad y antigüedad, ya que si bien la probabilidad de aparición en el futuro es baja, son datos que nos permiten conocer mejor a nuestro cliente.  

En el caso de los productos que son considedos montos fijos, si creemos prudente eliminarlos, ya que no es el objetivo de esta mentoría realizar una predicción respecto a ellos. 

In [54]:
df = df[~df['producto_naranja_movimiento'].isin(['AV','SM'])]

##### Fechas 

In [55]:
print('Cantidad de casos antes de Abril 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-04-01')]))
print('Cantidad de casos antes de Mayo 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-05-01')]))
print('Cantidad de casos antes de Junio 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-06-01')]))
print('Cantidad de casos antes de Julio 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-07-01')]))
print('Cantidad de casos antes de Agosto 2020:', len(df[df['fecha_cupon_movimiento'] < pd.to_datetime('2020-08-01')]))

Cantidad de casos antes de Abril 2020: 1
Cantidad de casos antes de Mayo 2020: 2
Cantidad de casos antes de Junio 2020: 3
Cantidad de casos antes de Julio 2020: 7
Cantidad de casos antes de Agosto 2020: 900


In [56]:
print('Cantidad de casos despues de Abril 2021:', len(df[df['fecha_cupon_movimiento'] > pd.to_datetime('2021-04-01')]))
print('Cantidad de casos despues de Mayo 2021:', len(df[df['fecha_cupon_movimiento'] > pd.to_datetime('2021-05-01')]))
print('Cantidad de casos despues de Junio 2021:', len(df[df['fecha_cupon_movimiento'] > pd.to_datetime('2021-06-01')]))

Cantidad de casos despues de Abril 2021: 6194
Cantidad de casos despues de Mayo 2021: 448
Cantidad de casos despues de Junio 2021: 0


A pesar de que más adelante se realizará un balanceo de registros en los diferentes meses, observamos que la cantidad de transacciones en los meses previos a Agosto del 2020, son demasiados bajos para ser consideramos en nuestros modelos de predicción. 

Haciendo un cálculo similar a los ultimos meses analizados, vemos que Mayo y Abril del 2021 están bien representados, por lo que se decide conservarlos.

In [57]:
df = df[df['fecha_cupon_movimiento'] > pd.to_datetime('2020-07-01')]

In [58]:
df.shape

(60523, 42)

#### 1.4 Variables categoricas (Reagrupación)

##### Estado Civil

In [59]:
df.estado_civil_descripcion.value_counts()

Solteroa               29292
Casadoa                23218
Divorciadoa             2443
Viudoa                  2193
Separacion de hecho     1333
Concubinoa              1308
Sin Datos                680
Novioa                    56
Name: estado_civil_descripcion, dtype: int64

Observamos que existe una alta frecuencia en las categorías Solteros y Casados, perdiendo representativadad en el resto de los estados civiles, por lo cual decidimos agrupar a la mismas en una misma categoría.

In [60]:
soltero = ['Solteroa']
casado = ['Casadoa']
otros = ['Divorciadoa','Viudoa','Concubinoa','Separacion de hecho','Novioa']
sin_datos = ['Sin Datos']

df.loc[df['estado_civil_descripcion'].str.contains('|'.join(soltero)),'estado_civil_cat'] = 'Soltero'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(casado)),'estado_civil_cat'] = 'Casado'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(otros)),'estado_civil_cat'] = 'Otros'
df.loc[df['estado_civil_descripcion'].str.contains('|'.join(sin_datos)),'estado_civil_cat'] = 'Sin_datos'

In [61]:
df.estado_civil_cat.value_counts()

Soltero      29292
Casado       23218
Otros         7333
Sin_datos      680
Name: estado_civil_cat, dtype: int64

##### Provincias por Regiones

In [62]:
df[['provincia']].groupby(by=['provincia']).size()\
                .to_frame()\
                .reset_index(level=['provincia'])\
                .rename(columns = {0:'Cantidad_Transacciones'})\
                .sort_values(by='Cantidad_Transacciones',ascending=False)

,provincia,Cantidad_Transacciones
5,CORDOBA,11992
0,BUENOS AIRES,9816
20,SANTA FE,4507
23,TUCUMAN,4417
12,MENDOZA,3852
4,CHUBUT,2570
3,CHACO,2331
6,CORRIENTES,2260
1,CAPITAL FEDERAL,1908
7,ENTRE RIOS,1824


Viendo las frecuencias por provincias, vamos a recategorizar sobre una nueva columna, las provincias por regiones.

In [63]:
dic_region = {'REGION NOROESTE': ['JUJUY','SALTA','TUCUMAN','CATAMARCA','SGO DEL ESTERO'],
              'REGION NORDESTE': ['CHACO','FORMOSA','CORRIENTES','MISIONES'], 
              'REGION PAMPEANA': ['CORDOBA','BUENOS AIRES','CAPITAL FEDERAL','ENTRE RIOS','LA PAMPA','SANTA FE'], 
              'REGION CUYO': ['SAN JUAN','SAN LUIS','LA RIOJA','MENDOZA'], 
              'REGION PATAGONIA': ['SANTA CRUZ','TIERRA DEL FUEGO','RIO NEGRO','NEUQUEN','CHUBUT'],
              'SIN DATOS': ['Sin Datos']}

df['region']= df['provincia']
for i in dic_region:
    df['region'] = df['region'].replace(dic_region[i], i)
df.head()

,dni_titular_movimiento,moneda_movimiento,id_comercio_movimiento,nombre_comercio_histo,numero_cupon_movimiento,debito_credito_movimiento,producto_naranja_movimiento,codigo_empresa_movimiento,tipo_producto_tarjeta_movimiento,plan_movimiento,...,rel_vivienda_descripcion_histo,anio_mes_cupon,monto_ajustado,nombre_comercio_concat,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,estado_civil_cat,region
0,0001686b52949b5461ffcbc766687e45031,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Propia,202008,5.52,INTERES POR MORA US,Sector_Empleado_Comercio,61,92,0,Otros,REGION PAMPEANA
1,000220fa96ec5af89817894033f8099c547,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Sin Datos,202008,15.68,INTERES POR MORA US,Sector_Sin_Datos,29,2,0,Sin_datos,REGION CUYO
2,0002be202de47dfae9cc2304d91161be595,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Otros,202008,5.46,INTERES POR MORA US,Sector_Sin_Datos,28,95,0,Soltero,REGION PAMPEANA
3,000e137d0af42e193be1ff670c00d4d1506,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Propia,202008,2.50,INTERES POR MORA US,Sector_Empleado_Comercio,40,151,0,Soltero,REGION PAMPEANA
4,0009d010e4faf69552a814a33832b185877,0,020099784,INTERES POR MORA US,200813,0,PL,1,0,1,...,Alquilada,202008,2.10,INTERES POR MORA US,Sector_Empleado_Comercio,36,87,0,Soltero,REGION PAMPEANA


In [64]:
df[['region']].groupby(by=['region']).size()

region
REGION CUYO          6981
REGION NORDESTE      6864
REGION NOROESTE     10303
REGION PAMPEANA     30319
REGION PATAGONIA     6056
dtype: int64

#### 1.5 Selección de columnas

Algunas columnas se eliminaron en analisis anteriores. Procedemos a seleccionar las columnas que mejor nos ayudaran a predecir la próxima compra para modelos de aprendizaje no supervisado, donde el objetivo es lograr la mejor caracterización del cliente tipo. 

* **dni_titular_movimiento: Se conserva por ser ID único de cliente**
* **moneda_movimiento: Se conserva para identificar consumos en moneda extranjera**
* id_comercio_movimiento: Se descarta porque conservamos comercio_cat que surge a partir de esta misma variable
* nombre_comercio_histo: Se descarta por usar comercio_cat
* numero_cupon_movimiento: Se descarta por no aportar informacion util a la predicción de la compra
* debito_credito_movimiento: Solo permite diferenciar movimientos positivos de negativos, informacion presente en la variable monto.
* **producto_naranja_movimiento: Se conserva**
* codigo_empresa_movimiento: Se descarta por no aportar informacion util a la predicción de la compra
* **tipo_producto_tarjeta_movimiento: Se conserva**
* plan_movimiento: Se descarta por no aportar informacion util a la predicción de la compra
* **fecha_cupon_movimiento: Se conserva**
* monto_compra_movimiento: Se descarta, ya que se conserva monto_ajustado, que es el resultado de su transformación
* importe_cuota_movimiento: Se descarta por su fuerte correlación con monto_compra_movimiento
* interes_movimiento: Se descarta por su falta de datos significativos
* cargo_adm_seguro_movimiento, cargo_otorgamiento_movimiento, cargo_seguro_vida_movimiento, cargo_administrativo_movimiento: Se descartan por tener mayoria de datos en 0.
* seleccionado_ng: Se descarta por no representar nada significativo
* codigo_contable_movimiento: Se descarta por su baja interpretabilidad
* local_venta_producto: Se descarta por aportar la misma información que id_comercio_movimiento
* marca_debito_automatico: Se descarta por no aportar informacion util a la predicción de la compra
* id_comercio_otras_marcas_movimiento: Se descarta por tener mayoria de datos en 0 y una correlación fuerte con id_comercio
* estado_civil_descripcion: Se descarta por recategorización
* **sexo_descripcion: Se conserva**  
* provincia: Se descarta por recategorizacion en regiones
* ciudad: Se descarta para tener un analisis geografico limitado hasta regiones
* **domicilio_codigo_postal:  Se conserva**  
* fecha_de_ingreso_histo: Se descarta porque su único objetivo era calcular la antigüedad del cliente
* fecha_presentacion_movimiento: Se descarta por su fuerte correlación con fecha_cupon_movimiento
* fecha_vto_cupon_movimiento: Se descarta por su fuerte correlación con fecha_cupon_movimiento
* cargo_descripcion_histo: Se descarta por recategorizacion
* **nivel_estudio_descripcion_histo: Se conserva**
* **rel_vivienda_descripcion_histo: Se conserva**

In [65]:
df_final = df[['dni_titular_movimiento', 'fecha_cupon_movimiento', 'moneda_movimiento', 'producto_naranja_movimiento', 'tipo_producto_tarjeta_movimiento',
               'anio_mes_cupon', 'sexo_descripcion', 'monto_ajustado',
               'cargo_sector_desc_hist', 'edad_cliente', 'antig_cliente', 'comercio_cat', 
               'estado_civil_cat', 'region','nivel_estudio_descripcion_histo','rel_vivienda_descripcion_histo',
               'domicilio_codigo_postal']]       

In [66]:
df_final.shape

(60523, 17)

In [67]:
df_final

,dni_titular_movimiento,fecha_cupon_movimiento,moneda_movimiento,producto_naranja_movimiento,tipo_producto_tarjeta_movimiento,anio_mes_cupon,sexo_descripcion,monto_ajustado,cargo_sector_desc_hist,edad_cliente,antig_cliente,comercio_cat,estado_civil_cat,region,nivel_estudio_descripcion_histo,rel_vivienda_descripcion_histo,domicilio_codigo_postal
0,0001686b52949b5461ffcbc766687e45031,2020-08-25,0,PL,0,202008,Hombre,5.52,Sector_Empleado_Comercio,61,92,0,Otros,REGION PAMPEANA,SECUNDARIOS,Propia,1766
1,000220fa96ec5af89817894033f8099c547,2020-08-25,0,PL,0,202008,Mujer,15.68,Sector_Sin_Datos,29,2,0,Sin_datos,REGION CUYO,Sin Datos,Sin Datos,5521
2,0002be202de47dfae9cc2304d91161be595,2020-08-25,0,PL,0,202008,Mujer,5.46,Sector_Sin_Datos,28,95,0,Soltero,REGION PAMPEANA,PRIMARIOS,Otros,7107
3,000e137d0af42e193be1ff670c00d4d1506,2020-08-25,0,PL,0,202008,Hombre,2.50,Sector_Empleado_Comercio,40,151,0,Soltero,REGION PAMPEANA,SECUNDARIOS,Propia,2200
4,0009d010e4faf69552a814a33832b185877,2020-08-25,0,PL,0,202008,Mujer,2.10,Sector_Empleado_Comercio,36,87,0,Soltero,REGION PAMPEANA,UNIVERSITARIOS,Alquilada,1019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124000,0002716f6d9c2130a8c88a0219de234d459,2021-05-14,0,PC,3,202105,Mujer,9661.01,Sector_No_Operativo,46,93,0,Soltero,REGION NOROESTE,PRIMARIOS,Propia,4181
124003,0001f61dd2845a7e653ebfdaf22dab3b373,2021-05-14,0,PC,3,202105,Mujer,20329.24,Sector_Empleado_Comercio,48,98,0,Casado,REGION PATAGONIA,SECUNDARIOS,Propia,8300
124005,0006c15ca823454b68c189da1344d9d7317,2021-05-14,0,PC,3,202105,Hombre,13210.67,Sector_No_Operativo,66,105,0,Casado,REGION NORDESTE,SECUNDARIOS,Propia,3503
124006,0000ab27a0ed815f947df8bcb834ff97975,2021-05-03,0,PC,3,202105,Hombre,121.73,Sector_Empleado_Comercio,74,140,0,Soltero,REGION NOROESTE,SECUNDARIOS,Propia,4200


#### 1.6 Codificación de variables 

In [68]:
# A efectos de corroborar la cantidad de columnas de nuestra matriz

print("sexo_descripcion", df_final.sexo_descripcion.nunique())
print("cargo_sector_desc_hist", df_final.cargo_sector_desc_hist.nunique())
print("estado_civil_cat", df_final.estado_civil_cat.nunique())
print("region", df_final.region.nunique())
print("comercio_cat", df_final.comercio_cat.nunique())
print("producto_naranja_movimiento", df_final.producto_naranja_movimiento.nunique())
print("moneda_movimiento", df_final.moneda_movimiento.nunique())
print("tipo_producto_tarjeta_movimiento", df_final.tipo_producto_tarjeta_movimiento.nunique())
print("nivel_estudio_descripcion_histo", df_final.nivel_estudio_descripcion_histo.nunique())
print("rel_vivienda_descripcion_histo", df_final.rel_vivienda_descripcion_histo.nunique()) 

sexo_descripcion 3
cargo_sector_desc_hist 8
estado_civil_cat 4
region 5
comercio_cat 10
producto_naranja_movimiento 10
moneda_movimiento 3
tipo_producto_tarjeta_movimiento 6
nivel_estudio_descripcion_histo 5
rel_vivienda_descripcion_histo 5


In [69]:
ordinal_ft = 'dni_titular_movimiento'
target = 'monto_ajustado'
#target_c = 'fg_aumentado'

# Features numericas
num_features = ['anio_mes_cupon', 'edad_cliente', 'antig_cliente', 'domicilio_codigo_postal','fecha_cupon_movimiento']

# Features categoricas de la transaccion
trans_ft = ['producto_naranja_movimiento', 'tipo_producto_tarjeta_movimiento', 'moneda_movimiento', 'comercio_cat']

# Features categoricas del cliente
client_ft = ['sexo_descripcion', 'cargo_sector_desc_hist', 'estado_civil_cat', 'region', 'nivel_estudio_descripcion_histo', 
             'rel_vivienda_descripcion_histo']

In [70]:
# Ordenamos por fecha
df = df.sort_values(by = ['fecha_cupon_movimiento'], ascending = True)
# Transformacion de fecha a numerica
df['fecha_cupon_movimiento'] = df['fecha_cupon_movimiento'].values.astype(float)/10**11
# Transformacion de tipo de moneda a string
df['moneda_movimiento'] = df['moneda_movimiento'].astype(str)

# Codificación
cat_transformer = OneHotEncoder(handle_unknown='ignore')

encoder = ColumnTransformer(
    transformers=[
        ('dni', 'drop', [ordinal_ft]),
        ('num', 'passthrough', num_features),
        ('trans', cat_transformer, trans_ft),
        ('client', cat_transformer, client_ft),
        ('target', 'passthrough', [target])])
df_enc = encoder.fit_transform(df)
cols = encoder.get_feature_names()

# Agregar feature ordinal al principio del dataframe

dff_enc = pd.DataFrame.sparse.from_spmatrix(df_enc, columns=[cols]).sparse.to_dense()
cols_o = np.hstack([[ordinal_ft],cols])
df_stack = np.hstack([df[[ordinal_ft]],dff_enc])
df_encode = pd.DataFrame(df_stack , columns=cols_o)

In [71]:
df_encode

,dni_titular_movimiento,anio_mes_cupon,edad_cliente,antig_cliente,domicilio_codigo_postal,fecha_cupon_movimiento,trans__x0_AX,trans__x0_EX,trans__x0_MC,trans__x0_PC,...,client__x4_SECUNDARIOS,client__x4_Sin Datos,client__x4_TERCIARIOS,client__x4_UNIVERSITARIOS,client__x5_Alquilada,client__x5_De familiar,client__x5_Otros,client__x5_Propia,client__x5_Sin Datos,monto_ajustado
0,00017c577769060500211670502411b5913,202007.0,62.0,79.0,3460.0,15937344.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-3445.56
1,0000c20705a45563f2ec6a53088c2a30090,202007.0,80.0,134.0,3016.0,15938208.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-707.0
2,000b0b6905ca172cb064331c76663c00147,202007.0,43.0,72.0,5584.0,15939072.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-344.07
3,000eb089db186a25b5ae4701ca1d297a941,202007.0,51.0,138.0,4430.0,15939936.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1467.39
4,000603de9fd38f5dcd0870949ce24d26089,202007.0,72.0,281.0,5521.0,15939936.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-400.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60518,0007d3573a085abc49ede83321981a7d760,202106.0,50.0,181.0,7600.0,16219008.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-9.0
60519,0001b49c252907291ca16195773ef104079,202105.0,49.0,6.0,4113.0,16219008.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,82.64
60520,0001b49c252907291ca16195773ef104079,202105.0,49.0,6.0,4113.0,16219008.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,423.62
60521,000a9fa6856d6d70da80b3320cbb7a58581,202105.0,33.0,5.0,4000.0,16219008.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2660.35


## 2. Agrupamiento/Balanceo 

### 2.1 Agrupamiento de transacciones por Cliente, Mes, Producto Naranja y Tipo de producto¶

Procedemos a agrupar las transacciones de los clientes por mes, producto y tipo de producto, debido a que el objeto de esta mentoría es predecir tendencias de consumo mensuales de los diferentes clientes. Aquellas variables que refieren a los datos personales, fueron tomadas por su valor único. 

In [74]:
df_encode.columns

Index(['dni_titular_movimiento', 'anio_mes_cupon', 'edad_cliente',
       'antig_cliente', 'domicilio_codigo_postal', 'fecha_cupon_movimiento',
       'trans__x0_AX', 'trans__x0_EX', 'trans__x0_MC', 'trans__x0_PC',
       'trans__x0_PL', 'trans__x0_PN', 'trans__x0_PP', 'trans__x0_TA',
       'trans__x0_VI', 'trans__x0_ZE', 'trans__x1_0', 'trans__x1_3',
       'trans__x1_4', 'trans__x1_22', 'trans__x1_32', 'trans__x1_42',
       'trans__x2_0', 'trans__x2_1', 'trans__x2_3', 'trans__x3_0',
       'trans__x3_1', 'trans__x3_2', 'trans__x3_3', 'trans__x3_4',
       'trans__x3_5', 'trans__x3_6', 'trans__x3_7', 'trans__x3_8',
       'trans__x3_9', 'client__x0_Hombre', 'client__x0_Mujer',
       'client__x0_Sin Datos', 'client__x1_Sector_Educativo',
       'client__x1_Sector_Empleado_Comercio', 'client__x1_Sector_Financiero',
       'client__x1_Sector_No_Operativo', 'client__x1_Sector_Operativo',
       'client__x1_Sector_Salud', 'client__x1_Sector_Seguridad',
       'client__x1_Sector_Sin_Dato

In [75]:
# ver que hacemos con fecha_cupon
#df_encode = df_encode.rename({'domicilio_codigo_postal': 'client__domicilio_codigo_postal'}, axis=1)
#df_encode

In [76]:
num_features

['anio_mes_cupon',
 'edad_cliente',
 'antig_cliente',
 'domicilio_codigo_postal',
 'fecha_cupon_movimiento']

In [77]:
#df_encode |= df_encode['domicilio_codigo_postal'].astype(int)

In [88]:
# Funciones de agregacion para cada columna
aggr = {} 
aggr.update(dict.fromkeys([x for x in cols if 'client' in x], 'max'))
aggr.update(dict.fromkeys([x for x in cols if 'trans' in x], 'sum'))
#aggr.update(dict.fromkeys([x for x in num_features], 'max'))
aggr.update({target:'sum'})

# Convertimos las columnas categoricas de la transaccion a numericas para poder sumarizarlas
df_encode[num_features + [x for x in cols if 'trans' in x]] = \
df_encode[num_features + [x for x in cols if 'trans' in x]].apply(pd.to_numeric)

# Agrupamiento
group = ['dni_titular_movimiento', 'anio_mes_cupon']

df_mes = df_encode.groupby(group).agg(aggr).reset_index() # edad y antiguedad

In [89]:
df_mes

,dni_titular_movimiento,anio_mes_cupon,edad_cliente,antig_cliente,client__x0_Hombre,client__x0_Mujer,client__x0_Sin Datos,client__x1_Sector_Educativo,client__x1_Sector_Empleado_Comercio,client__x1_Sector_Financiero,...,trans__x3_1,trans__x3_2,trans__x3_3,trans__x3_4,trans__x3_5,trans__x3_6,trans__x3_7,trans__x3_8,trans__x3_9,monto_ajustado
0,000000b5aea2c9ea7cc155f6ebcef97f826,202008.0,46.0,225.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.72
1,000000b5aea2c9ea7cc155f6ebcef97f826,202009.0,47.0,226.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.61
2,000000b5aea2c9ea7cc155f6ebcef97f826,202010.0,47.0,227.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.46
3,000000b5aea2c9ea7cc155f6ebcef97f826,202011.0,47.0,228.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.34
4,000000b5aea2c9ea7cc155f6ebcef97f826,202012.0,47.0,229.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7506,000f0b73ebfa002a79a0642b82e87919904,202101.0,64.0,21.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11616.45
7507,000f0b73ebfa002a79a0642b82e87919904,202102.0,64.0,22.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12566.68
7508,000f0b73ebfa002a79a0642b82e87919904,202103.0,64.0,23.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2663.08
7509,000f0b73ebfa002a79a0642b82e87919904,202104.0,64.0,24.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5660.76


### 2.2 Balanceo de meses 

Corroboramos que existen algunos clientes que no tienen movimientos todos los meses, por lo cuál procedemos a generar esas filas para balancear el dataset

In [ ]:
df_mes[['dni_titular_movimiento','anio_mes_cupon']].groupby(['dni_titular_movimiento']).size().value_counts()

In [ ]:
array_mes = df_mes.anio_mes_cupon.sort_values().unique()
array_dni = df_mes.dni_titular_movimiento.unique()

dic = {'dni': [], 'mes': []}
for dni in array_dni:
    for mes in array_mes:
        dic['dni'].append(dni)
        dic['mes'].append(mes)

df_mes_imp = pd.merge(pd.DataFrame(dic), 
                      df_mes, 
                      left_on=['dni','mes'], 
                      right_on=['dni_titular_movimiento','anio_mes_cupon'], 
                      how='left') \
                .drop(columns=['dni_titular_movimiento','anio_mes_cupon'])
df_mes_imp = df_mes_imp.rename(columns={'dni': 'dni_titular_movimiento','mes': 'anio_mes_cupon'})
df_mes_imp

In [ ]:
df_mes_imp[['dni_titular_movimiento','anio_mes_cupon']].groupby(['dni_titular_movimiento']).size().value_counts()

In [ ]:
# Imputamos 0 en las columnas de la transacción que tienen nan
cols = df_mes_imp.columns.values
cols_zero = [x for x in cols if 'trans' in x]
cols_zero.append('monto_ajustado')
df_mes_imp[cols_zero] = df_mes_imp[cols_zero].fillna(0)

# Imputamos lo valores propios del cliente
imputer = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
cols_client = [x for x in cols if 'client_' in x]
df_mes_imp[cols_client] = imputer.fit_transform(df_mes_imp[cols_client].values)

#Ordenamos por dni y mes
df_mes_imp = df_mes_imp.sort_values(by=['dni_titular_movimiento','anio_mes_cupon']).reset_index().drop(columns=['index'])

# Imputamos antigüedad
for dni in array_dni:
    min_mes = df_mes_imp[(df_mes_imp['dni_titular_movimiento'] == dni) & (~df_mes_imp.antig_cliente.isna())]['anio_mes_cupon'].min()
    min_ant = df_mes_imp[(df_mes_imp['dni_titular_movimiento'] == dni) & (~df_mes_imp.antig_cliente.isna())]['antig_cliente'].min()
    for mes in array_mes:
        num_res = array_mes.tolist().index(mes) - array_mes.tolist().index(min_mes)
        antig_cli = min_ant + num_res
        if antig_cli < 0:
            df_mes_imp.loc[((df_mes_imp['dni_titular_movimiento'] == dni) & (df_mes_imp['anio_mes_cupon'] == mes)), 'antig_cliente'] = 0
        else:
            df_mes_imp.loc[((df_mes_imp['dni_titular_movimiento'] == dni) & (df_mes_imp['anio_mes_cupon'] == mes)), 'antig_cliente'] = antig_cli

In [ ]:
# Imputación Edad
df_mes_imp_eb = df_mes_imp[['dni_titular_movimiento', 'anio_mes_cupon', 'edad_cliente']].copy()
for dni in array_dni:
    edad = df_mes_imp_eb[df_mes_imp['dni_titular_movimiento'] == dni]['edad_cliente'].fillna(method='backfill')
    df_mes_imp_eb.loc[((df_mes_imp['dni_titular_movimiento'] == dni) & (df_mes_imp['edad_cliente'].isna())), 'edad_cliente'] = edad
df_mes_imp_eb = df_mes_imp_eb.rename(columns={'edad_cliente': 'edad_b'})

df_mes_imp_ef = df_mes_imp[['dni_titular_movimiento', 'anio_mes_cupon', 'edad_cliente']].copy()
for dni in array_dni:
    edad = df_mes_imp_ef[df_mes_imp['dni_titular_movimiento'] == dni]['edad_cliente'].fillna(method='ffill')
    df_mes_imp_ef.loc[((df_mes_imp['dni_titular_movimiento'] == dni) & (df_mes_imp['edad_cliente'].isna())), 'edad_cliente'] = edad
df_mes_imp_ef = df_mes_imp_ef.rename(columns={'edad_cliente': 'edad_f'})

df_mes_imp = df_mes_imp.merge(df_mes_imp_eb,
                             left_on=['dni_titular_movimiento', 'anio_mes_cupon'],
                             right_on=['dni_titular_movimiento', 'anio_mes_cupon']
                            )
df_mes_imp = df_mes_imp.merge(df_mes_imp_ef,
                             left_on=['dni_titular_movimiento', 'anio_mes_cupon'],
                             right_on=['dni_titular_movimiento', 'anio_mes_cupon']
                            )
df_mes_imp.loc[df_mes_imp['edad_cliente'].isna(), 'edad_cliente'] = df_mes_imp['edad_b']
df_mes_imp.loc[df_mes_imp['edad_cliente'].isna(), 'edad_cliente'] = df_mes_imp['edad_f']

df_mes_imp.drop(columns=['edad_b','edad_f'], inplace=True)

In [ ]:
df_mes = df_mes_imp.copy()

# 3. Creación de Nuevos Atributos

### 3.1 Creación de variable categórica

Creamos una columna que indica si un cliente ha aumentado su consumo personal mes a mes. El mes inicial para cada cliente se inicializa en 0.

Cabe destacar que a la hora de agrupar los movimientos en cada mes, se considera la fecha de cierre de la tarjeta, por lo cual cada mes comprende desde el dia 25 del mes anterior hasta el dia 24 del corriente, inclusive.

In [ ]:
df_dni_mes_mon = df_mes[['dni_titular_movimiento', 'anio_mes_cupon','monto_ajustado']] \
                        .groupby(['dni_titular_movimiento', 'anio_mes_cupon']).sum('monto_ajustado') \
                        .reset_index() \
                        .sort_values(by=['dni_titular_movimiento', 'anio_mes_cupon'])
df_dni_mes_mon

In [ ]:
# Agregación de la nueva columna seteada en 0.
# En la iteración, se le cambia el valor a 1 en caso de que cumpla con las condiciones preestablecidas.

df_dni_mes_mon['fg_aumentado'] = 0
for i in range(1,len(df_dni_mes_mon)):
    if (df_dni_mes_mon.iloc[i]['dni_titular_movimiento'] == df_dni_mes_mon.iloc[i-1]['dni_titular_movimiento']) \
        & (df_dni_mes_mon.iloc[i]['anio_mes_cupon'] > df_dni_mes_mon.iloc[i-1]['anio_mes_cupon']):
            var_mes = df_dni_mes_mon.iloc[i]['monto_ajustado'] - df_dni_mes_mon.iloc[i-1]['monto_ajustado']
            if (var_mes > (abs(df_dni_mes_mon.iloc[i-1]['monto_ajustado'])*0.1)):
                    df_dni_mes_mon.iloc[i,3] = 1

In [ ]:
df_dni_mes_mon.head(20)

In [ ]:
df_dni_mes_mon = df_dni_mes_mon.drop(columns=['monto_ajustado'])
df_dni_mes_mon = df_dni_mes_mon.rename(columns={'dni_titular_movimiento': 'dni_titular_movimiento_c', 'anio_mes_cupon': 'anio_mes_cupon_c'})
df_dni_mes_mon

In [ ]:
df_obj = df_mes.merge( df_dni_mes_mon, 
             left_on=['dni_titular_movimiento', 'anio_mes_cupon'], 
             right_on=['dni_titular_movimiento_c', 'anio_mes_cupon_c']) \
        .drop(columns= ['dni_titular_movimiento_c','anio_mes_cupon_c'])

In [ ]:
df_obj

In [ ]:
df_obj.shape

### 3.1 Conversión de variable domicilio_código_postal

In [ ]:
df_obj.domicilio_codigo_postal.describe()